In [1]:
import os
import time
from datetime import datetime, timedelta

In [2]:
os.add_dll_directory('C:\\Program Files\\IBM\\SQLLIB\\BIN')
# conectar a la base de datos IBM Db2
import ibm_db

In [3]:
from queries import get_academic_history, get_socioeconomico, tasa_reprobacion, dificultad_materia
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
def format_time(seconds):
    """Convierte segundos a formato HH:MM:SS"""
    return str(timedelta(seconds=int(seconds)))

def print_progress_bar(current, total, bar_length=30):
    """Imprime una barra de progreso"""
    percent = float(current) * 100 / total
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    return f'[{bar}] {percent:.1f}% ({current}/{total})'

In [5]:
print("="*80)
print("INICIO DE DESCARGA DE DATOS ACADÉMICOS")
print("="*80)
print(f"Fecha y hora de inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

INICIO DE DESCARGA DE DATOS ACADÉMICOS
Fecha y hora de inicio: 2025-09-29 09:46:43


In [6]:
# Tiempo de inicio total
start_time_total = time.time()

print("\n📡 Conectando con la base de datos...")
start_time_conn = time.time()


📡 Conectando con la base de datos...


In [7]:
conn_str = (
    'DATABASE=' + os.getenv('DATABASE') + 
    ';HOSTNAME=' + os.getenv('HOSTNAME') + 
    ';PORT=' + os.getenv('PORT') + 
    ';PROTOCOL=TCPIP;UID=' + os.getenv('USERNAME_DB') + 
    ';PWD=' + os.getenv('PASSWORD_DB') + ';'
)

print(repr(conn_str))
conn = ibm_db.connect(conn_str, '', '')

'DATABASE=SAAC;HOSTNAME=192.168.254.53;PORT=50000;PROTOCOL=TCPIP;UID=espol;PWD=espol123;'


In [8]:
end_time_conn = time.time()
connection_time = end_time_conn - start_time_conn

if conn:
    print(f"✅ Conexión exitosa - Tiempo: {connection_time:.2f} segundos")
else:
    print("❌ Error al conectar")
    exit(1)

✅ Conexión exitosa - Tiempo: 107.02 segundos


In [9]:
# Cargar datos de materias
print("\n📁 Cargando lista de materias...")
start_time_load = time.time()
df_materias = pd.read_csv("./data/materias/tasa_reprobacion_dificultad.csv")
end_time_load = time.time()
load_time = end_time_load - start_time_load

print(f"✅ Materias cargadas - Tiempo: {load_time:.2f} segundos")
print(f"📊 Total de materias en archivo: {len(df_materias)}")


📁 Cargando lista de materias...
✅ Materias cargadas - Tiempo: 0.01 segundos
📊 Total de materias en archivo: 741


In [10]:
# Filtrar materias a procesar
list_not_grades = ["PORTAFOLIO"]
materias_to_process = []

for i in df_materias.iterrows():
    if i[1]["MATERIA_x"] in list_not_grades or "MATERIA INTEGRADORA" in i[1]["MATERIA_x"]:
        continue
    materias_to_process.append(i[1])

In [11]:
total_materias = len(materias_to_process)
print(f"🎯 Materias a procesar: {total_materias}")

# Parámetros de configuración
year = "2024"
semester = "2S"
print(f"📅 Período: {year} - {semester}")

🎯 Materias a procesar: 706
📅 Período: 2024 - 2S


In [12]:
print("\n" + "="*80)
print("INICIANDO PROCESAMIENTO DE MATERIAS")
print("="*80)

# Variables para estadísticas
total_students_processed = 0
failed_materias = []
processing_times = []


INICIANDO PROCESAMIENTO DE MATERIAS


In [13]:
# Procesar cada materia
for counter, materia_data in enumerate(materias_to_process, 1):
    print(f"\n{'='*60}")
    print(f"PROCESANDO MATERIA {counter}/{total_materias}")
    print(f"{'='*60}")
    
    cod_materia = materia_data["CODIGOMATERIA"].strip()
    materia_name = materia_data["MATERIA_x"]
    
    print(f"📚 Código: {cod_materia}")
    print(f"📖 Nombre: {materia_name}")
    print(f"⏱️  {print_progress_bar(counter-1, total_materias)}")
    
    # Tiempo de inicio para esta materia
    start_time_materia = time.time()
    
    try:
        # Query 1: Historial académico
        print(f"\n🔍 Ejecutando query de historial académico...")
        start_time_query1 = time.time()
        
        query = get_academic_history(cod_materia, year, semester)
        stmt_select = ibm_db.exec_immediate(conn, query)
        
        data_list = []
        result = ibm_db.fetch_assoc(stmt_select)
        while result:
            result["COD_MATERIA_ACAD_MO"] = result["COD_MATERIA_ACAD_MO"].strip()
            data_list.append(result)
            result = ibm_db.fetch_assoc(stmt_select)
        
        df = pd.DataFrame(data_list)
        end_time_query1 = time.time()
        query1_time = end_time_query1 - start_time_query1
        
        num_students = len(df)
        print(f"✅ Query 1 completada - Estudiantes encontrados: {num_students} - Tiempo: {query1_time:.2f}s")
        
        if num_students == 0:
            print(f"⚠️  No hay estudiantes para esta materia, saltando...")
            continue
        
        # Query 2: Datos socioeconómicos
        print(f"🔍 Ejecutando query socioeconómico...")
        start_time_query2 = time.time()
        
        query = get_socioeconomico(df["COD_ESTUDIANTE"].to_list())
        stmt_select = ibm_db.exec_immediate(conn, query)
        
        data_list = []
        result = ibm_db.fetch_assoc(stmt_select)
        while result:
            data_list.append(result)
            result = ibm_db.fetch_assoc(stmt_select)
        
        df_socio = pd.DataFrame(data_list)
        end_time_query2 = time.time()
        query2_time = end_time_query2 - start_time_query2
        
        socio_records = len(df_socio)
        print(f"✅ Query 2 completada - Registros socioeconómicos: {socio_records} - Tiempo: {query2_time:.2f}s")
        
        # Merge y guardado
        print(f"🔗 Realizando merge y guardando archivo...")
        start_time_merge = time.time()
        
        df["COD_ESTUDIANTE"] = df["COD_ESTUDIANTE"].astype(str).str.strip()
        df_socio["CODESTUDIANTE"] = df_socio["CODESTUDIANTE"].astype(str).str.strip()
        
        df_merge = pd.merge(df, df_socio, left_on='COD_ESTUDIANTE', right_on="CODESTUDIANTE", how='left')
        
        filename = f"./data/materias/{cod_materia}_{year}_{semester}.csv"
        df_merge.to_csv(filename, index=False)
        
        end_time_merge = time.time()
        merge_time = end_time_merge - start_time_merge
        
        # Tiempo total para esta materia
        end_time_materia = time.time()
        materia_time = end_time_materia - start_time_materia
        processing_times.append(materia_time)
        
        print(f"✅ Archivo guardado: {filename}")
        print(f"💾 Merge y guardado - Tiempo: {merge_time:.2f}s")
        print(f"📊 Registros finales: {len(df_merge)}")
        
        # Estadísticas de la materia
        total_students_processed += num_students
        coverage = (socio_records / num_students * 100) if num_students > 0 else 0
        
        print(f"\n📈 RESUMEN MATERIA:")
        print(f"   • Estudiantes académicos: {num_students}")
        print(f"   • Registros socioeconómicos: {socio_records}")
        print(f"   • Cobertura socioeconómica: {coverage:.1f}%")
        print(f"   • Query 1: {query1_time:.2f}s")
        print(f"   • Query 2: {query2_time:.2f}s")
        print(f"   • Merge/Guardado: {merge_time:.2f}s")
        print(f"   • Tiempo total materia: {format_time(materia_time)}")
        
        # Estimación de tiempo restante
        if len(processing_times) >= 3:  # Solo después de procesar al menos 3 materias
            avg_time_per_materia = sum(processing_times) / len(processing_times)
            remaining_materias = total_materias - counter
            estimated_remaining_time = avg_time_per_materia * remaining_materias
            
            print(f"⏰ Tiempo promedio por materia: {format_time(avg_time_per_materia)}")
            print(f"⏳ Tiempo estimado restante: {format_time(estimated_remaining_time)}")
            
            estimated_finish = datetime.now() + timedelta(seconds=estimated_remaining_time)
            print(f"🏁 Finalización estimada: {estimated_finish.strftime('%H:%M:%S')}")
        # break
    except Exception as e:
        print(f"❌ ERROR procesando {cod_materia}: {str(e)}")
        failed_materias.append({
            'codigo': cod_materia,
            'nombre': materia_name,
            'error': str(e)
        })


PROCESANDO MATERIA 199/706
📚 Código: ACUG1035
📖 Nombre: ACUICULTURA ORNAMENTAL
⏱️  [████████----------------------] 28.0% (198/706)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 2 - Tiempo: 12.55s
🔍 Ejecutando query socioeconómico...
✅ Query 2 completada - Registros socioeconómicos: 2 - Tiempo: 8.22s
🔗 Realizando merge y guardando archivo...
✅ Archivo guardado: ./data/materias/ACUG1035_2024_2S.csv
💾 Merge y guardado - Tiempo: 0.02s
📊 Registros finales: 2

📈 RESUMEN MATERIA:
   • Estudiantes académicos: 2
   • Registros socioeconómicos: 2
   • Cobertura socioeconómica: 100.0%
   • Query 1: 12.55s
   • Query 2: 8.22s
   • Merge/Guardado: 0.02s
   • Tiempo total materia: 0:00:20

PROCESANDO MATERIA 200/706
📚 Código: ACUG1036
📖 Nombre: ANÁLISIS DE DATOS ACUÍCOLAS
⏱️  [████████----------------------] 28.2% (199/706)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 5 - Tiempo: 12.38s
🔍 Ejecutando qu

In [14]:
# Cerrar conexión
print(f"\n🔌 Cerrando conexión a la base de datos...")
ibm_db.close(conn)

# Tiempo total
end_time_total = time.time()
total_time = end_time_total - start_time_total



🔌 Cerrando conexión a la base de datos...


In [15]:
print("\n" + "="*80)
print("RESUMEN FINAL DEL PROCESAMIENTO")
print("="*80)
print(f"🏁 Fecha y hora de finalización: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️  Tiempo total de ejecución: {format_time(total_time)}")
print(f"📊 Materias procesadas exitosamente: {total_materias - len(failed_materias)}/{total_materias}")
print(f"👥 Total estudiantes procesados: {total_students_processed:,}")



RESUMEN FINAL DEL PROCESAMIENTO
🏁 Fecha y hora de finalización: 2025-09-29 13:04:33
⏱️  Tiempo total de ejecución: 3:17:46
📊 Materias procesadas exitosamente: 476/706
👥 Total estudiantes procesados: 23,366



RESUMEN FINAL DEL PROCESAMIENTO

🏁 Fecha y hora de finalización: 2025-09-26 19:40:40
⏱️  Tiempo total de ejecución: 3:42:18
📊 Materias procesadas exitosamente: 691/706
👥 Total estudiantes procesados: 28,639

In [16]:
if processing_times:
    avg_time = sum(processing_times) / len(processing_times)
    min_time = min(processing_times)
    max_time = max(processing_times)
    
    print(f"\n📈 ESTADÍSTICAS DE TIEMPO:")
    print(f"   • Tiempo promedio por materia: {format_time(avg_time)}")
    print(f"   • Tiempo mínimo: {format_time(min_time)}")
    print(f"   • Tiempo máximo: {format_time(max_time)}")
    print(f"   • Velocidad promedio: {total_students_processed/total_time:.1f} estudiantes/segundo")


📈 ESTADÍSTICAS DE TIEMPO:
   • Tiempo promedio por materia: 0:00:26
   • Tiempo mínimo: 0:00:14
   • Tiempo máximo: 0:00:57
   • Velocidad promedio: 2.0 estudiantes/segundo


In [17]:
if failed_materias:
    print(f"\n❌ MATERIAS CON ERRORES ({len(failed_materias)}):")
    for failed in failed_materias:
        print(f"   • {failed['codigo']} - {failed['nombre'][:50]}")
        print(f"     Error: {failed['error']}")


❌ MATERIAS CON ERRORES (230):
   • AGRG1022 - AGRICULTURA DE PRECISIÓN
     Error: 'CODESTUDIANTE'
   • ALIG1033 - DISEÑO DE PLANTAS ALIMENTARIAS
     Error: 'CODESTUDIANTE'
   • MCTG1013 - DISEÑO MECATRÓNICO
     Error: 'utf-8' codec can't decode byte 0xe1 in position 115: invalid continuation byte
   • MCTG1014 - DISEÑO Y MANUFACTURA ASISTIDA POR COMPUTADORA
     Error: 'utf-8' codec can't decode byte 0xf3 in position 37: invalid continuation byte
   • MCTG1015 - DISEÑO Y SIMULACIÓN DE MÁQUINAS
     Error: 'utf-8' codec can't decode byte 0xf3 in position 37: invalid continuation byte
   • MCTG1016 - HIDRÁULICA Y NEUMÁTICA
     Error: 'utf-8' codec can't decode byte 0xf3 in position 37: invalid continuation byte
   • MCTG1017 - INTRODUCCIÓN A LA MANUFACTURA
     Error: 'utf-8' codec can't decode byte 0xf3 in position 37: invalid continuation byte
   • MCTG1019 - SISTEMAS BIOMECATRÓNICOS
     Error: 'utf-8' codec can't decode byte 0xf3 in position 37: invalid continuation byte
   • MC

❌ MATERIAS CON ERRORES (15):
   • ACUG1045 - PRODUCCIÓN ACUÍCOLA II
     Error: 'CODESTUDIANTE'
   • ALIG1033 - DISEÑO DE PLANTAS ALIMENTARIAS
     Error: 'CODESTUDIANTE'
   • CCPG1058 - SISTEMAS DE INFORMACIÓN APLICADOS A LOGÍSTICA
     Error: 'CODESTUDIANTE'
   • ESTG1056 - DATOS NO ESTRUCTURADOS
     Error: 'CODESTUDIANTE'
   • GEOG1027 - DISEÑO DE CAMPO GEOLÓGICO
     Error: 'CODESTUDIANTE'
   • GEOG1037 - INGENIERÍA GEOLÓGICA
     Error: 'CODESTUDIANTE'
   • INDG1061 - HSEQ EN LA INDUSTRIA HIDROCARBURÍFERA
     Error: 'CODESTUDIANTE'
   • MING1031 - PLANEAMIENTO Y DISEÑO MINERO
     Error: 'CODESTUDIANTE'
   • MTRG1027 - DISEÑO DE MATERIALES COMPUESTOS
     Error: 'CODESTUDIANTE'
   • MTRG1041 - NANOTECNOLOGÍA Y NANOMATERIALES
     Error: 'CODESTUDIANTE'
   • PETG1022 - INGENIERÍA DE PRODUCCIÓN II
     Error: 'CODESTUDIANTE'
   • PETG1023 - INGENIERÍA DE YACIMIENTOS I
     Error: 'CODESTUDIANTE'
   • PETG1024 - INGENIERÍA DE YACIMIENTOS II
     Error: 'CODESTUDIANTE'
   • PETG1028 - SIMULACIÓN DE RESERVORIOS
     Error: 'CODESTUDIANTE'
   • PETG1034 - EVALUACIÓN DE FORMACIONES II
     Error: 'CODESTUDIANTE'

In [18]:

print(f"\n✅ Procesamiento completado!")
print("="*80)


✅ Procesamiento completado!
